In [8]:
import json
import re
import random
from tqdm import tqdm
from typing import Optional, Dict, Sequence, List
# import argparse

def extract_last_num(text: str) -> float:
    text = re.sub(r"(\d),(\d)", "\g<1>\g<2>", text)  # 处理形如 123,456
    res = re.findall(r"(\d+(\.\d+)?)", text)  # 匹配 123456.789
    if len(res) > 0:
        num_str = res[-1][0]
        return float(num_str)
    else:
        return 0.0
def check(key, truth, predict):
    # correct_samples  = {task:[] for task in tasks}


    if key in ['cycle', 'connectivity', 'hamilton', 'substructure', 'bipartite']:
       
        if 'yes' in truth.lower() and 'yes' in predict.split('###')[-1].lower():
            # correct_samples[key].append(v)
            return True
        elif 'no' in truth.lower() and 'no' in predict.split('###')[-1].lower():
            return True
        return False
    
    elif key in ['flow', 'shortest', 'triplet']:
      
        t_num = extract_last_num(truth)
        p_num = extract_last_num(predict.split('###')[-1])
        if abs(t_num - p_num) < 1e-2:
            return True
        return False
                
    elif key == 'topology':
        
        pre = predict.split('###')[-1].strip(' ')
        
        truth = truth.split('###')[-1].strip(' ')

        if truth in pre or pre in truth:
            return True
        return False
    
    


In [28]:
with open('/cpfs/user/chennuo/CN/Graph_RFT_Data/gpt4data/gpt4_gsm8knlg_sample3_output.json') as f:
    datas = f.readlines()
print(len(datas))

tasks = ['cycle', 'connectivity', 'hamilton', 'substructure', 'bipartite', 'flow', 'shortest', 'triplet', 'topology']
correct_samples  = {task:[] for task in tasks}
all_samples  = {task:[] for task in tasks}
temp = 0
select_samples = []
math_samples = []
for data in datas:
    
    data = json.loads(data)
        
    task = data['task']
    
    if task in ['4','5', '6', '7', '8','9']:
        temp += 1
        math_samples.append(data)
        continue
    if check(task, data['response'], data['dv3_response']):
        if data['query'] not in  correct_samples[task]:
            correct_samples[task].append(data['query'])
            select_samples.append(data)
            
    if data['query'] not in all_samples[task]:
        all_samples[task].append(data['query'])
        
with open('/cpfs/user/chennuo/CN/Graph_RFT_Data/gpt4data/gpt4_generate_nlg.json', 'w' ) as writer:
    for sample in select_samples:
        writer.write(json.dumps(sample, ensure_ascii=False) + '\n')


with open('/cpfs/user/chennuo/CN/Graph_RFT_Data/gpt4data/gpt4_generate_math.json', 'w' ) as writer:
    for sample in math_samples:
        writer.write(json.dumps(sample, ensure_ascii=False) + '\n')

67324


In [26]:
for key, value in all_samples.items():
    print(key)
    print(len(value))

cycle
3000
connectivity
3000
hamilton
3000
substructure
1130
bipartite
2991
flow
3000
shortest
2999
triplet
3000
topology
2973


In [32]:
len(select_samples)

16127

In [27]:
for key, value in correct_samples.items():
    print(key)
    total = all_samples[key]
    print(len(value)/len(total))

# all_samples

cycle
0.9406666666666667
connectivity
0.9246666666666666
hamilton
0.742
substructure
0.827433628318584
bipartite
0.6756937479103978
flow
0.07533333333333334
shortest
0.49783261087029007
triplet
0.9186666666666666
topology
0.2939791456441305


In [30]:
correct_samples['triplet'][-1]

'Find the maximum sum of the weights of three interconnected nodes. In an undirected graph, [i, k] means that node i has the weight k. (i,j) means that node i and node j are connected with an undirected edge. Given a graph, you need to output the maximum sum of the weights of three interconnected nodes. \nQ: The nodes are numbered from 0 to 13, weights of nodes are: [0, 1] [1, 5] [2, 6] [3, 10] [4, 8] [5, 3] [6, 10] [7, 5] [8, 3] [9, 6] [10, 9] [11, 9] [12, 7] [13, 6], and the edges are: (1, 7) (1, 3) (3, 7) (4, 5) (8, 9). What is the maximum sum of the weights of three nodes?'

In [21]:
import json
with open('/cpfs/user/chennuo/CN/Graph_RFT_Data/gpt4data/gpt4_generate_nlg.json') as f:
    datas = f.readlines()
refine_datas = []
temp = 0
for data in datas:
    data = json.loads(data)
    new_data = dict()
    new_data['query'] = data['query']
    new_data['task'] = data['task']
    new_data['response'] = data['response']
    response = data['dv3_response'].split('A:')[-1].split('\n\n')[-1]
    if '###' not in response:
        response +=  data['response'] +'.'
    if 'print' in response or 'return' in response or 'def' in response:
        temp += 1
        continue
    new_data['CoT_response'] = response
    refine_datas.append(new_data)

with open('/cpfs/user/chennuo/CN/Graph_RFT_Data/gpt4data/graph_source_data_v1.json', 'w' ) as writer:
    for sample in refine_datas:
        writer.write(json.dumps(sample, ensure_ascii=False) + '\n')

In [22]:
import random
random.sample(refine_datas, 10)[0]

{'query': 'Determine whether or not a graph is bipartite. In a directed graph, (i->j) means that node i and node j are connected with an directed edge from node i to node j. Given a graph, you need to output Yes or No, indicating whether the graph is bipartite. \nQ: The nodes are numbered from 0 to 8, and the edges are: (0->7) (0->3) (0->6) (1->7) (1->4) (1->5) (4->8) (4->5) (5->6) (5->8). Is this graph bipartite?',
 'task': 'bipartite',
 'response': '### No',
 'CoT_response': ' To examine whether the given graph is bipartite, we try to assign the nodes to two different sets, A and B, such that no two adjacent nodes belong to the same set. \nThe graph has nodes from 0 to 8 and edges like (0->7), (1->4), and (5->8). \nWe can start by putting node 0 in set A and node 7 in set B, since they are connected by an edge. \nThen, we can put node 3 and node 6 in set B, since they are connected to node 0 by edges. \nNext, we can put node 1 in set A, since it is connected to node 7 by an edge. \nN

In [25]:

PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}


with open('/cpfs/user/chennuo/CN/Graph-Reasoning-LLM/datasets/data/graph_source_data_v1_dsformat.json', 'w' ) as writer:
    for sample in refine_datas:
        
        sample['prompt'] = PROMPT_DICT["prompt_no_input"].format(instruction=sample['query'])
        sample['chosen'] = sample['CoT_response']
        sample['rejected'] = 'I do not know'
        writer.write(json.dumps(sample, ensure_ascii=False) + '\n')